In [250]:
import os
import shutil
import random
import json
import pretty_midi

import numpy as np
import tensorflow as tf
from tensorflow import keras

import pydub
from pydub import AudioSegment

import librosa
import random

import subprocess

import parse

In [ ]:
# load data list
data_list = {}
train_list = []
test_list = []

with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\400-100-dataset\list.json', 'r') as f:
    data_list = json.load(f)

train_list = data_list['train']
test_list = data_list['test']

va_list = {}

In [18]:
# load model
model = keras.models.load_model(r'C:\Users\Arnold\Desktop\4560\CNN\best_crnn')

In [7]:
# dir
train_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\train'
output_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output'
checkpoint_file = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\cat-mel_2bar_small\train'

In [116]:
# run code in CMD and will print the output of IN TIME ref.: https://lumeng.blog.csdn.net/article/details/104845611
def runCMD(cmd):
    screenData = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    while True:
        line = screenData.stdout.readline()
        print(line.decode('gbk').strip("b'"))
        if line == b'' or subprocess.Popen.poll(screenData) == 0:
            screenData.stdout.close()
            break

In [117]:
# split midi in to time fragment
def split_midi_by_time(midi_data,frag_start,frag_finish):
    new_midi = midi_data

    for i in range(len(new_midi.instruments)):
        inst = new_midi.instruments[i]
        for n in range(len(inst.notes)-1,-1,-1):
            note = inst.notes[n]
            if ((note.end<frag_start) or (note.start>frag_finish)):
                inst.notes.remove(inst.notes[n])
        for n in range(len(inst.notes)-1,-1,-1):
            note = inst.notes[n]
            note.start = max(note.start - frag_start,0)
            note.end = min(note.end - frag_start, frag_finish - frag_start)

    return new_midi

In [216]:
# interpolate music
def interpolate_music(train_dir,output_dir,music1,music2,frag1,frag2):
    midi_data = midi_data = pretty_midi.PrettyMIDI(train_dir + '\\' + music1 + '.mp3.mid')
    new_midi = split_midi_by_time(midi_data,(frag1/2+15),(frag1/2+15)+4)
    interMusic_1 = output_dir + '\\'+ music1 + '_trim.mid'
    new_midi.write(interMusic_1)
    
    midi_data = midi_data = pretty_midi.PrettyMIDI(train_dir + '\\' + music2 + '.mp3.mid')
    new_midi = split_midi_by_time(midi_data,(frag2/2+15),(frag2/2+15)+4)
    interMusic_2 = output_dir + '\\'+ music2 + '_trim.mid'
    new_midi.write(interMusic_2)
    
    music_interpolate = r'music_vae_generate \
    --config={config} \
    --checkpoint_file={checkpoint_file} \
    --mode=interpolate \
    --num_outputs={num_outputs} \
    --input_midi_1={interMusic_1} \
    --input_midi_2={interMusic_2} \
    --output_dir={output_dir}'

    runCMD(music_interpolate.
              format(config=config,checkpoint_file=checkpoint_file,num_outputs=1,
                    interMusic_1=interMusic_1, interMusic_2=interMusic_2,output_dir=output_dir))

    output_num = 0
    for parent,dirnames, filenames in os.walk(output_dir):
        for filename in filenames:
            if filename.find("cat-mel_2bar_small")!=-1:
                newname = "{music1}-{frag1}th-{frag14}th_{music2}-{frag2}th-{frag24}th_{output_num}-of-2".
                    format(music1=music1,frag1=frag1,frag14=frag1+8,music2=music2,frag2=frag2,frag24=frag2+8,
                        output_num=output_num)
                new_midi_name = os.path.join(parent,newname + ".mid")
                new_wav_name = os.path.join(parent,newname + ".wav")
                
                os.rename(os.path.join(parent,filename),new_midi_name)
                output_num+=1
                
                runCMD(r'TiMidity\timidity.exe {midifile} -Ow -o {wavfile}'.format(
                    midifile = new_midi_name, wavfile = new_wav_name))
        
        
    os.remove(interMusic_1)
    os.remove(interMusic_2)
    
    return newname[:-7]

In [224]:
# run the model on the output.wav
def evaluate_wav_output(model,output_dir,inter_result):
    result = []
    for parent, dirnames, filenames in os.walk(output_dir,  followlinks=True):
        for filename in filenames:
            if (filename.endswith(".wav") and (filename.find(inter_result)!=-1)):
                index = os.path.splitext(filename)[0]
#                 print(os.path.join(output_dir, filename))
                wavsong = AudioSegment.from_wav(os.path.join(output_dir, filename))

                one_data = []
                test_data = []

                # mel freq
                for i in range(4):
                    outputpath = os.path.join(output_dir,index+'-'+str(i)+".wav")
                    wavsong[(500*i):(500+500*i)].export(outputpath, format="wav")
                    y, sr = librosa.load(outputpath, sr=None)
                    os.remove(outputpath)

                    fragment = np.array(
                                    np.log(
                                        librosa.feature.melspectrogram(
                                            y, sr, n_mels=128,n_fft=int(sr*0.5), hop_length=int(sr*1)
                                        )
                                    ).transpose())
                    fragment = fragment[np.newaxis,:]

                    if one_data==[]:
                        one_data = fragment.copy()                    
                    else:
                        one_data = np.vstack([one_data,fragment.copy()])

                # self repliction
                for i in range(15):
                    if test_data==[]:
                        test_data = one_data.copy()                    
                    else:
                        test_data = np.vstack([one_data,test_data.copy()])                    

                # crnn test
                test_data = test_data.reshape(1,60,128,1)
                predict_va = model.predict(test_data)
                result.append([sum(sum(predict_va[0])/60),sum(sum(predict_va[1])/60)])
    return result

In [256]:
# write result into csv file
def write_result(record_csv,inter_result,eva_result):
    aver_eva_result = [0,0]
    for i in range(len(eva_result)):
        for j in range(2):
            aver_eva_result[j] += eva_result[i][j]
    aver_eva_result = [i/len(eva_result) for i in aver_eva_result]
    print(aver_eva_result)
    
    inter_result_format = "{music1}-{frag1}th-{frag14}th_{music2}-{frag2}th-{frag24}th"
    parsed = parse.parse(inter_result_format,inter_result)
    print(parsed)

In [257]:
write_result("","1583-49th-57th_1216-39th-47th",[[100, 0], [50, 0],[30,0]])

[60.0, 0.0]
<Result () {'music1': '1583', 'frag1': '49', 'frag14': '57', 'music2': '1216', 'frag2': '39', 'frag24': '47'}>


In [221]:
testtimes = 1
for i in range(testtimes):
    [music_1,music_2] = random.sample(train_list,2)
    frag_1 = random.randint(0,52)
    frag_2 = random.randint(0,52)
    inter_result = interpolate_music(train_dir,output_dir,music_1,music_2,frag_1,frag_2)
    eva_result = evaluate_wav_output(model,output_dir,inter_result)
    print(inter_result,eva_result)
    #write_result(record_csv,inter_result,eva_result)


Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1103 21:34:32.646222 21840 music_vae_generate.py:145] Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

2 valid inputs extracted from `C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\1583_trim.mid`. Outputting these potential inputs as `C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\cat-mel_2bar_small_input1-extractions_2020-11-03_213432-*-of-002.mid`. Call script again with one of these instead.


Playing C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\1583-49th-57th_1216-39th-47th_0-of-2.mid

MIDI file: C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\1583-49th-57th_1216-39th-47th_0-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~8 seconds

Notes cut: 0

Notes lost totally: 0


Playing C:\Users\Arnold\D

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


1583-49th-57th_1216-39th-47th [(-0.1850691585405002, -0.04721060219890205), (-0.24064462626120076, -0.03388033243300015)]
